# 4. 对话机器人示例 Chatbot Example

本节课，我们来熟悉整个课程都会用到的聊天机器人，同时进一步了解工具的定义和使用，课后还要多和机器人聊天哦！

## 一、导入必要的库

In [1]:
# 首先导入必要的库
import arxiv # 本节材料的主要来源是公开在arxiv上的论文
import json
import os
from typing import List
from dotenv import load_dotenv # 如果使用导入.env文件的方式记录密钥需要加载这个包
import openai

## 二、工具代码

In [2]:
PAPER_DIR = "papers"

第一个使用的工具是用来搜索和我们设定主题相关的arxiv论文的，最后搜索到的内容会以固定的形式保存在一个json文件中(title, authors, summary, paper url and the publication date)
但是并不会下载这些文章，json文件会保存在PAPER_DIR目录下。

In [3]:
def search_papers(topic: str, max_results: int = 5) -> List[str]:
    # 使用arxiv来搜索论文 
    client = arxiv.Client()

    # 搜索和我们设定主题最相关的论文
    search = arxiv.Search(
        query = topic,
        max_results = max_results,
        sort_by = arxiv.SortCriterion.Relevance
    )

    papers = client.results(search)
    
    # 为我们当前的主题创建目录
    path = os.path.join(PAPER_DIR, topic.lower().replace(" ", "_"))
    os.makedirs(path, exist_ok=True)
    
    file_path = os.path.join(path, "papers_info.json")

    # 尝试加载已经找到的论文信息
    try:
        with open(file_path, "r") as json_file:
            papers_info = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        papers_info = {}

    # 处理找到的每篇论文并添加到paper_info中
    paper_ids = []
    for paper in papers:
        paper_ids.append(paper.get_short_id())
        paper_info = {
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'summary': paper.summary,
            'pdf_url': paper.pdf_url,
            'published': str(paper.published.date())
        }
        papers_info[paper.get_short_id()] = paper_info
    
    # Save updated papers_info to json file
    with open(file_path, "w") as json_file:
        json.dump(papers_info, json_file, indent=2)
    
    print(f"Results are saved in: {file_path}")
    
    return paper_ids

In [4]:
search_papers("AI") # 设定想要搜索的论文主题，会返回对应的论文编号

Results are saved in: papers\ai\papers_info.json


['2409.12922v1',
 '2406.11563v3',
 '2402.07632v3',
 '2211.05075v1',
 '2403.15481v2']

In [5]:
def extract_info(paper_id: str) -> str:
    """
    在目录中寻找某篇特定文章的信息，
    需要使用对应的文章编号（上节返回的内容）来查找，
    会返回完整的json列表。
    """
 
    for item in os.listdir(PAPER_DIR):
        item_path = os.path.join(PAPER_DIR, item)
        if os.path.isdir(item_path):
            file_path = os.path.join(item_path, "papers_info.json")
            if os.path.isfile(file_path):
                try:
                    with open(file_path, "r") as json_file:
                        papers_info = json.load(json_file)
                        if paper_id in papers_info:
                            return json.dumps(papers_info[paper_id], indent=2)
                except (FileNotFoundError, json.JSONDecodeError) as e:
                    print(f"Error reading {file_path}: {str(e)}")
                    continue
    
    return f"There's no saved information related to paper {paper_id}."

In [6]:
extract_info('2409.12922v1') # 可以修改为任意需要的文章编号

'{\n  "title": "AI Thinking: A framework for rethinking artificial intelligence in practice",\n  "authors": [\n    "Denis Newman-Griffis"\n  ],\n  "summary": "Artificial intelligence is transforming the way we work with information\\nacross disciplines and practical contexts. A growing range of disciplines are\\nnow involved in studying, developing, and assessing the use of AI in practice,\\nbut these disciplines often employ conflicting understandings of what AI is and\\nwhat is involved in its use. New, interdisciplinary approaches are needed to\\nbridge competing conceptualisations of AI in practice and help shape the future\\nof AI use. I propose a novel conceptual framework called AI Thinking, which\\nmodels key decisions and considerations involved in AI use across disciplinary\\nperspectives. The AI Thinking model addresses five practice-based competencies\\ninvolved in applying AI in context: motivating AI use in information processes,\\nformulating AI methods, assessing availa

In [7]:
# 这里是一个完整的工具定义，包含两个工具示例
tools = [
    {
        "name": "search_papers", # 工具命名
        "description": "Search for papers on arXiv based on a topic and store their information.", # 描述工具的作用
        "input_schema": { # 规定输入的格式
            "type": "object",
            "properties": {
                "topic": {
                    "type": "string",
                    "description": "The topic to search for"
                }, 
                "max_results": {
                    "type": "integer",
                    "description": "Maximum number of results to retrieve",
                    "default": 5
                }
            },
            "required": ["topic"]
        }
    },
    {
        "name": "extract_info",
        "description": "Search for information about a specific paper across all topic directories.",
        "input_schema": {
            "type": "object",
            "properties": {
                "paper_id": {
                    "type": "string",
                    "description": "The ID of the paper to look for"
                }
            },
            "required": ["paper_id"]
        }
    }
]

In [8]:
mapping_tool_function = {
    "search_papers": search_papers,
    "extract_info": extract_info
}

# 编写工具实现的逻辑
def execute_tool(tool_name, tool_args):
    
    result = mapping_tool_function[tool_name](**tool_args)

    if result is None:
        result = "操作已经完成但没有任何结果。"
        
    elif isinstance(result, list):
        result = ', '.join(result)
        
    elif isinstance(result, dict):
        # Convert dictionaries to formatted JSON strings
        result = json.dumps(result, indent=2)
    
    else:
        # For any other type, convert using str()
        result = str(result)
    return result

## 三、聊天机器人代码

In [9]:
from openai import OpenAI

# 指向 DeepSeek 的 API 地址因为deepseek兼容openai的调用方式，可以直接使用openai的库
client = OpenAI(
    api_key="sk-hjjjmxuugiiybzffneckhpmngqeqzojtxyeywvhxaniilfcn",  # 替换为你的密钥
    base_url="https://api.siliconflow.cn/v1"  # 这里可以使用任意一家支持deepseek调用的公司对方会提供调用的地址
)


In [10]:
def process_query(query):
    
    messages = [{'role': 'user', 'content': query}]
    
    response = client.messages.create(max_tokens = 2024,
                                  model = 'deepseek-ai/DeepSeek-V2.5', # 这里替换成你需要的模型
                                  tools = tools,
                                  messages = messages)
    
    process_query = True
    while process_query:
        assistant_content = []

        for content in response.content:
            if content.type == 'text':
                
                print(content.text)
                assistant_content.append(content)
                
                if len(response.content) == 1:
                    process_query = False
            
            elif content.type == 'tool_use':
                
                assistant_content.append(content)
                messages.append({'role': 'assistant', 'content': assistant_content})
                
                tool_id = content.id
                tool_args = content.input
                tool_name = content.name
                print(f"Calling tool {tool_name} with args {tool_args}")
                
                result = execute_tool(tool_name, tool_args)
                messages.append({"role": "user", 
                                  "content": [
                                      {
                                          "type": "tool_result",
                                          "tool_use_id": tool_id,
                                          "content": result
                                      }
                                  ]
                                })
               
                response = client.chat.completions.create(
                                    model = "deepseek-ai/DeepSeek-V2.5", # 这里替换成你需要的模型
                                    tools = tools,
                                    messages = messages
                )
                
                if len(response.content) == 1 and response.content[0].type == "text":
                    print(response.content[0].text)
                    process_query = False

In [11]:
def chat_loop():
    print("输入你的问题，或者输入“退出”来退出")
    while True:
        try:
            query = input("\n问题: ").strip()
            if query.lower() == '退出':
                break
    
            process_query(query)
            print("\n")
        except Exception as e:
            print(f"\nError: {str(e)}")

In [12]:
chat_loop()

输入你的问题，或者输入“退出”来退出



问题:  退出
